Подключаем GoogleDrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Устанавливаем OpenJDK

In [ ]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

Устанавливаем переменные окружения, необходимые для скачивания дистрибутива Spark

In [ ]:
import os
os.environ['APACHE_SPARK_DISTR_NAME'] = 'spark-3.0.2-bin-hadoop2.7'
os.environ['APACHE_SPARK_URL'] = 'https://downloads.apache.org/spark/spark-3.0.2/spark-3.0.2-bin-hadoop2.7.tgz'

Скачиваем и распаковываем архив с дистрибутивом Spark

In [ ]:
![ ! -d $APACHE_SPARK_DISTR_NAME ] \
   && wget -q $APACHE_SPARK_URL \
   && tar xf $APACHE_SPARK_DISTR_NAME.tgz \
   && rm -f $APACHE_SPARK_DISTR_NAME.tgz

Устанавливаем переменные окружения, необходимые для работы дистрибутива Spark

In [ ]:
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-11-openjdk-amd64'
os.environ['SPARK_HOME'] = '/content/spark-3.0.2-bin-hadoop2.7'

Устанавливаем пакет findspark

In [ ]:
!pip install findspark > /dev/null

In [ ]:
f = open('./drive/My Drive/text.txt', 'r')
f.read()

'["http","https","amp","rt","t","c","the",\'able\', \'about\', \'above\', \'abroad\', \'according\', \'accordingly\', \'across\', \'actually\', \'adj\', \'after\', \'afterwards\', \'again\', \'against\', \'ago\', \'ahead\', "ain\'t", \'all\', \'allow\', \'allows\', \'almost\', \'alone\', \'along\', \'alongside\', \'already\', \'also\', \'although\', \'always\', \'am\', \'amid\', \'amidst\', \'among\', \'amongst\', \'an\', \'and\', \'another\', \'any\', \'anybody\', \'anyhow\', \'anyone\', \'anything\', \'anyway\', \'anyways\', \'anywhere\', \'apart\', \'appear\', \'appreciate\', \'appropriate\', \'are\', "aren\'t", \'around\', \'as\', "a\'s", \'aside\', \'ask\', \'asking\', \'associated\', \'at\', \'available\', \'away\', \'awfully\', \'back\', \'backward\', \'backwards\', \'be\', \'became\', \'because\', \'become\', \'becomes\', \'becoming\', \'been\', \'before\', \'beforehand\', \'begin\', \'behind\', \'being\', \'believe\', \'below\', \'beside\', \'besides\', \'best\', \'better\', \

Подключаем пакет findspark

In [ ]:
import findspark
findspark.init()

Подключаем пакет SparkSession

In [ ]:
from pyspark.sql import SparkSession

Создаём новую сессию

In [ ]:
spark = SparkSession.builder.master('local').appName("HPC.Lab1").getOrCreate()

Загружаем данные из исходного датасета в дата-фрейм main_df и показываем

In [ ]:
data = spark.read.csv(r"./drive/My Drive/data.csv", header=True)
data.show()

+---+--------------------+--------------------+--------------------+--------+--------------------+--------------------+-------------+
| ID|               TITLE|                 URL|           PUBLISHER|CATEGORY|               STORY|            HOSTNAME|    TIMESTAMP|
+---+--------------------+--------------------+--------------------+--------+--------------------+--------------------+-------------+
|  1|Fed official says...|http://www.latime...|   Los Angeles Times|       b|ddUyU0VZz0BRneMio...|     www.latimes.com|1394470370698|
|  2|Fed's Charles Plo...|http://www.livemi...|            Livemint|       b|ddUyU0VZz0BRneMio...|    www.livemint.com|1394470371207|
|  3|US open: Stocks f...|http://www.ifamag...|        IFA Magazine|       b|ddUyU0VZz0BRneMio...| www.ifamagazine.com|1394470371550|
|  4|Fed risks falling...|http://www.ifamag...|        IFA Magazine|       b|ddUyU0VZz0BRneMio...| www.ifamagazine.com|1394470371793|
|  5|Fed's Plosser: Na...|http://www.moneyn...|           Mone

In [ ]:
data = data.na.drop()

In [ ]:
from pyspark.sql.functions import col
data.groupBy("category") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------------------+------+
|            category| count|
+--------------------+------+
|                   e|152127|
|                   b|115934|
|                   t|108236|
|                   m| 45616|
|         Us Magazine|    31|
|    Contactmusic.com|    20|
|           GossipCop|    20|
|         Complex.com|    12|
|            CBS News|    12|
|            HipHopDX|    11|
|The Hollywood Gossip|    11|
| We Got This Covered|    10|
|  HeadlinePlanet.com|    10|
|             Gamepur|     8|
|Consequence of Sound|     7|
|            Wetpaint|     7|
|   WorstPreviews.com|     7|
|          TooFab.com|     7|
|        The Escapist|     6|
|    Reality TV World|     5|
+--------------------+------+
only showing top 20 rows



In [ ]:
data = data.where(col("CATEGORY").isin(["e", "b", "t", "m"]))
data.groupBy("category") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------+------+
|category| count|
+--------+------+
|       e|152127|
|       b|115934|
|       t|108236|
|       m| 45616|
+--------+------+



In [ ]:
data.printSchema()

root
 |-- ID: string (nullable = true)
 |-- TITLE: string (nullable = true)
 |-- URL: string (nullable = true)
 |-- PUBLISHER: string (nullable = true)
 |-- CATEGORY: string (nullable = true)
 |-- STORY: string (nullable = true)
 |-- HOSTNAME: string (nullable = true)
 |-- TIMESTAMP: string (nullable = true)



In [ ]:
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.classification import LogisticRegression
# regular expression tokenizer
regexTokenizer = RegexTokenizer(inputCol="TITLE", outputCol="words", pattern="\\W")
# stop words
add_stopwords = ["http","https","amp","rt","t","c","the",'able', 'about', 'above', 'abroad', 'according', 'accordingly', 'across', 'actually', 'adj', 'after', 'afterwards', 'again', 'against', 'ago', 'ahead', "ain't", 'all', 'allow', 'allows', 'almost', 'alone', 'along', 'alongside', 'already', 'also', 'although', 'always', 'am', 'amid', 'amidst', 'among', 'amongst', 'an', 'and', 'another', 'any', 'anybody', 'anyhow', 'anyone', 'anything', 'anyway', 'anyways', 'anywhere', 'apart', 'appear', 'appreciate', 'appropriate', 'are', "aren't", 'around', 'as', "a's", 'aside', 'ask', 'asking', 'associated', 'at', 'available', 'away', 'awfully', 'back', 'backward', 'backwards', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before', 'beforehand', 'begin', 'behind', 'being', 'believe', 'below', 'beside', 'besides', 'best', 'better', 'between', 'beyond', 'both', 'brief', 'but', 'by', 'came', 'can', 'cannot', 'cant', "can't", 'caption', 'cause', 'causes', 'certain', 'certainly', 'changes', 'clearly', "c'mon", 'co', 'co.', 'com', 'come', 'comes', 'concerning', 'consequently', 'consider', 'considering', 'contain', 'containing', 'contains', 'corresponding', 'could', "couldn't", 'course', "c's", 'currently', 'dare', "daren't", 'definitely', 'described', 'despite', 'did', "didn't", 'different', 'directly', 'do', 'does', "doesn't", 'doing', 'done', "don't", 'down', 'downwards', 'during', 'each', 'edu', 'eg', 'eight', 'eighty', 'either', 'else', 'elsewhere', 'end', 'ending', 'enough', 'entirely', 'especially', 'et', 'etc', 'even', 'ever', 'evermore', 'every', 'everybody', 'everyone', 'everything', 'everywhere', 'ex', 'exactly', 'example', 'except', 'fairly', 'far', 'farther', 'few', 'fewer', 'fifth', 'first', 'five', 'followed', 'following', 'follows', 'for', 'forever', 'former', 'formerly', 'forth', 'forward', 'found', 'four', 'from', 'further', 'furthermore', 'get', 'gets', 'getting', 'given', 'gives', 'go', 'goes', 'going', 'gone', 'got', 'gotten', 'greetings', 'had', "hadn't", 'half', 'happens', 'hardly', 'has', "hasn't", 'have', "haven't", 'having', 'he', "he'd", "he'll", 'hello', 'help', 'hence', 'her', 'here', 'hereafter', 'hereby', 'herein', "here's", 'hereupon', 'hers', 'herself', "he's", 'hi', 'him', 'himself', 'his', 'hither', 'hopefully', 'how', 'howbeit', 'however', 'hundred', "i'd", 'ie', 'if', 'ignored', "i'll", "i'm", 'immediate', 'in', 'inasmuch', 'inc', 'inc.', 'indeed', 'indicate', 'indicated', 'indicates', 'inner', 'inside', 'insofar', 'instead', 'into', 'inward', 'is', "isn't", 'it', "it'd", "it'll", 'its', "it's", 'itself', "i've", 'just', 'k', 'keep', 'keeps', 'kept', 'know', 'known', 'knows', 'last', 'lately', 'later', 'latter', 'latterly', 'least', 'less', 'lest', 'let', "let's", 'like', 'liked', 'likely', 'likewise', 'little', 'look', 'looking', 'looks', 'low', 'lower', 'ltd', 'made', 'mainly', 'make', 'makes', 'many', 'may', 'maybe', "mayn't", 'me', 'mean', 'meantime', 'meanwhile', 'merely', 'might', "mightn't", 'mine', 'minus', 'miss', 'more', 'moreover', 'most', 'mostly', 'mr', 'mrs', 'much', 'must', "mustn't", 'my', 'myself', 'name', 'namely', 'nd', 'near', 'nearly', 'necessary', 'need', "needn't", 'needs', 'neither', 'never', 'neverf', 'neverless', 'nevertheless', 'new', 'next', 'nine', 'ninety', 'no', 'nobody', 'non', 'none', 'nonetheless', 'noone', 'no-one', 'nor', 'normally', 'not', 'nothing', 'notwithstanding', 'novel', 'now', 'nowhere', 'obviously', 'of', 'off', 'often', 'oh', 'ok', 'okay', 'old', 'on', 'once', 'one', 'ones', "one's", 'only', 'onto', 'opposite', 'or', 'other', 'others', 'otherwise', 'ought', "oughtn't", 'our', 'ours', 'ourselves', 'out', 'outside', 'over', 'overall', 'own', 'particular', 'particularly', 'past', 'per', 'perhaps', 'placed', 'please', 'plus', 'possible', 'presumably', 'probably', 'provided', 'provides', 'que', 'quite', 'qv', 'rather', 'rd', 're', 'really', 'reasonably', 'recent', 'recently', 'regarding', 'regardless', 'regards', 'relatively', 'respectively', 'right', 'round', 'said', 'same', 'saw', 'say', 'saying', 'says', 'second', 'secondly', 'see', 'seeing', 'seem', 'seemed', 'seeming', 'seems', 'seen', 'self', 'selves', 'sensible', 'sent', 'serious', 'seriously', 'seven', 'several', 'shall', "shan't", 'she', "she'd", "she'll", "she's", 'should', "shouldn't", 'since', 'six', 'so', 'some', 'somebody', 'someday', 'somehow', 'someone', 'something', 'sometime', 'sometimes', 'somewhat', 'somewhere', 'soon', 'sorry', 'specified', 'specify', 'specifying', 'still', 'sub', 'such', 'sup', 'sure', 'take', 'taken', 'taking', 'tell', 'tends', 'th', 'than', 'thank', 'thanks', 'thanx', 'that', "that'll", 'thats', "that's", "that've", 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'thence', 'there', 'thereafter', 'thereby', "there'd", 'therefore', 'therein', "there'll", "there're", 'theres', "there's", 'thereupon', "there've", 'these', 'they', "they'd", "they'll", "they're", "they've", 'thing', 'things', 'think', 'third', 'thirty', 'this', 'thorough', 'thoroughly', 'those', 'though', 'three', 'through', 'throughout', 'thru', 'thus', 'till', 'to', 'together', 'too', 'took', 'toward', 'towards', 'tried', 'tries', 'truly', 'try', 'trying', "t's", 'twice', 'two', 'un', 'under', 'underneath', 'undoing', 'unfortunately', 'unless', 'unlike', 'unlikely', 'until', 'unto', 'up', 'upon', 'upwards', 'us', 'use', 'used', 'useful', 'uses', 'using', 'usually', 'v', 'value', 'various', 'versus', 'very', 'via', 'viz', 'vs', 'want', 'wants', 'was', "wasn't", 'way', 'we', "we'd", 'welcome', 'well', "we'll", 'went', 'were', "we're", "weren't", "we've", 'what', 'whatever', "what'll", "what's", "what've", 'when', 'whence', 'whenever', 'where', 'whereafter', 'whereas', 'whereby', 'wherein', "where's", 'whereupon', 'wherever', 'whether', 'which', 'whichever', 'while', 'whilst', 'whither', 'who', "who'd", 'whoever', 'whole', "who'll", 'whom', 'whomever', "who's", 'whose', 'why', 'will', 'willing', 'wish', 'with', 'within', 'without', 'wonder', "won't", 'would', "wouldn't", 'yes', 'yet', 'you', "you'd", "you'll", 'your', "you're", 'yours', 'yourself', 'yourselves', "you've", 'zero', 'a', "how's", 'i', "when's", "why's", 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'j', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'uucp', 'w', 'x', 'y', 'z', 'I', 'www', 'amount', 'bill', 'bottom', 'call', 'computer', 'con', 'couldnt', 'cry', 'de', 'describe', 'detail', 'due', 'eleven', 'empty', 'fifteen', 'fifty', 'fill', 'find', 'fire', 'forty', 'front', 'full', 'give', 'hasnt', 'herse', 'himse', 'interest', 'itse”', 'mill', 'move', 'myse”', 'part', 'put', 'show', 'side', 'sincere', 'sixty', 'system', 'ten', 'thick', 'thin', 'top', 'twelve', 'twenty', 'abst', 'accordance', 'act', 'added', 'adopted', 'affected', 'affecting', 'affects', 'ah', 'announce', 'anymore', 'apparently', 'approximately', 'aren', 'arent', 'arise', 'auth', 'beginning', 'beginnings', 'begins', 'biol', 'briefly', 'ca', 'date', 'ed', 'effect', 'et-al', 'ff', 'fix', 'gave', 'giving', 'heres', 'hes', 'hid', 'home', 'id', 'im', 'immediately', 'importance', 'important', 'index', 'information', 'invention', 'itd', 'keys', 'kg', 'km', 'largely', 'lets', 'line', "'ll", 'means', 'mg', 'million', 'ml', 'mug', 'na', 'nay', 'necessarily', 'nos', 'noted', 'obtain', 'obtained', 'omitted', 'ord', 'owing', 'page', 'pages', 'poorly', 'possibly', 'potentially', 'pp', 'predominantly', 'present', 'previously', 'primarily', 'promptly', 'proud', 'quickly', 'ran', 'readily', 'ref', 'refs', 'related', 'research', 'resulted', 'resulting', 'results', 'run', 'sec', 'section', 'shed', 'shes', 'showed', 'shown', 'showns', 'shows', 'significant', 'significantly', 'similar', 'similarly', 'slightly', 'somethan', 'specifically', 'state', 'states', 'stop', 'strongly', 'substantially', 'successfully', 'sufficiently', 'suggest', 'thered', 'thereof', 'therere', 'thereto', 'theyd', 'theyre', 'thou', 'thoughh', 'thousand', 'throug', 'til', 'tip', 'ts', 'ups', 'usefully', 'usefulness', "'ve", 'vol', 'vols', 'wed', 'whats', 'wheres', 'whim', 'whod', 'whos', 'widely', 'words', 'world', 'youd', 'youre'
] 
stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered").setStopWords(add_stopwords)
# bag of words count
countVectors = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=10000, minDF=5)

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
label_stringIdx = StringIndexer(inputCol = "CATEGORY", outputCol = "label")
pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors, label_stringIdx])
# Fit the pipeline to training documents.
pipelineFit = pipeline.fit(data)
pipelineFit
dataset = pipelineFit.transform(data)
dataset.show(5)

+---+--------------------+--------------------+-----------------+--------+--------------------+-------------------+-------------+--------------------+--------------------+--------------------+-----+
| ID|               TITLE|                 URL|        PUBLISHER|CATEGORY|               STORY|           HOSTNAME|    TIMESTAMP|               words|            filtered|            features|label|
+---+--------------------+--------------------+-----------------+--------+--------------------+-------------------+-------------+--------------------+--------------------+--------------------+-----+
|  1|Fed official says...|http://www.latime...|Los Angeles Times|       b|ddUyU0VZz0BRneMio...|    www.latimes.com|1394470370698|[fed, official, s...|[fed, official, w...|(10000,[40,98,331...|  1.0|
|  2|Fed's Charles Plo...|http://www.livemi...|         Livemint|       b|ddUyU0VZz0BRneMio...|   www.livemint.com|1394470371207|[fed, s, charles,...|[fed, charles, pl...|(10000,[52,75,98,...|  1.0|
|  3|

In [ ]:
# set seed for reproducibility
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 295372
Test Dataset Count: 126541


In [ ]:
lr = LogisticRegression(maxIter=5, regParam=0.3, elasticNetParam=0)
lrModel = lr.fit(trainingData)
predictions = lrModel.transform(testData)
predictions.filter(predictions['prediction'] == 0) \
    .select("TITLE","CATEGORY","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

+------------------------------+--------+------------------------------+-----+----------+
|                         TITLE|CATEGORY|                   probability|label|prediction|
+------------------------------+--------+------------------------------+-----+----------+
|'How I Met Your Mother' Ser...|       e|[0.9938736875109319,0.00186...|  0.0|       0.0|
|ACM Awards 2014: Blake Shel...|       e|[0.9936239157065867,0.00175...|  0.0|       0.0|
|Kim Kardashian, Lamar Odom ...|       e|[0.992842836723842,0.002733...|  0.0|       0.0|
|Tony Awards 2014: Neil Patr...|       e|[0.9920767927677879,0.00315...|  0.0|       0.0|
|Courtney Love booed onstage...|       e|[0.9919576377354484,0.00196...|  0.0|       0.0|
|Jay Z, Beyoncé Laugh At Kan...|       e|[0.9919211340179123,0.00267...|  0.0|       0.0|
|BET Awards 2014 in Video: B...|       e|[0.9915938018770264,0.00298...|  0.0|       0.0|
|Horror Movie 'Deliver Us Fr...|       e|[0.9914383280160927,0.00282...|  0.0|       0.0|
|Rihanna, 

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

0.9250962868587319